In [1]:
import cv2 as cv
import numpy as np
import random
import os
import math

In [2]:
def click_corners(img, objpoints, imgpoints, objp, window, n_rows=8, n_cols=6):
    """
    get 4 corners from user clicks (in order) and use linear interpolation to get the
    other inner points.
    """

    order = ["TL", "BL", "TR", "BR"]
    box_corners = []

    # function to handle clicks on the image
    def click_event(event, x, y, flags, param):

        box_corners, img, window = param

        # check for left mouse clicks
        if event == cv.EVENT_LBUTTONDOWN and len(box_corners) < 4:

            box_corners.append((x, y))

            # display clicked points
            # on the image window
            font = cv.FONT_HERSHEY_SIMPLEX
            cv.putText(
                img, order[len(box_corners) - 1], (x, y), font, 1, (255, 0, 0), 2
            )
            cv.imshow(window, img)

    cv.namedWindow(window, cv.WINDOW_NORMAL)
    cv.imshow(window, img)
    cv.setMouseCallback(window, click_event, [box_corners, img, window])
    # wait for a key to be pressed to exit
    # print("click on the 4 corners, then press any key.")
    cv.waitKey(0)

    tl, bl, tr, br = np.array(box_corners, dtype=np.float32)

    # get first and last columns of points with linear interpolation
    first_col = np.linspace(tl, bl, n_rows)
    last_col = np.linspace(tr, br, n_rows)

    # get rest corner points by linearly interpolating the two columns
    all_points = np.vstack(
        [np.linspace(first_col[i], last_col[i], n_cols) for i in range(n_rows)]
    )
    corners = all_points.reshape(-1, 1, 2)

    objpoints.append(objp)
    imgpoints.append(corners)

    cv.destroyWindow(window)
    cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
    cv.imshow("Interpolated Chessboard Corners", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
# path = "./data/cam3/intrinsics.avi"
# cam = cv.VideoCapture(path)

# if not cam.isOpened():
#     print("Error: Could not open video file.")


# j = 0
# while True:

#     ret, frame = cam.read()
#     if ret:
#         j += 1

#         if j in [200, 230, 260, 280, 300, 320, 410, 530, 660, 700, 800]:
#             # [1, 30, 60, 80, 100, 120, 160, 180, 200, 230, 260, 300]
#             out_path = f"cam3_image{j}.jpg"
#             cv.imwrite(out_path, frame)
#             print(f"Saved {out_path}")

#         if j >= 1000:
#             break
#     else:
#         print("failed to grab frame")
#         break

# cam.release()

In [ ]:
def calibrate(video=1, n_rows=8, n_cols=6):

    vid = {
        1: [1, 30, 60, 80, 100, 120, 160, 180, 200, 230, 260, 300],
        2: [1, 30, 60, 80, 100, 120, 160, 180, 200, 230, 260, 300],
        3: [200, 230, 260, 280, 300, 320, 410, 530, 660, 700, 800],
        4: [1, 30, 60, 80, 100, 120, 160, 180, 200, 230, 260, 300],
    }

    objp = np.zeros((n_rows * n_cols, 3), np.float32)

    objp[:, :2] = np.mgrid[0:n_rows, 0:n_cols].T.reshape(-1, 2)

    objp = objp * 115

    # Lists to store object and image points from all images.

    objpoints = []  # 3d point in real world space

    imgpoints = []  # 2d points in image plane.


    for fr in vid[video]:

        frame = f"cam{video}_image{fr}.jpg"

        img = cv.imread(frame)

        if img is None:

            continue

        window = frame

        click_corners(
            img,
            objpoints,
            imgpoints,
            objp,
            window=window,
            n_rows=n_rows,
            n_cols=n_cols,
        )

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(

        objpoints, imgpoints, gray.shape[::-1], None, None
    )

    return ret, mtx, dist, rvecs, tvecs, objpoints, imgpoints

In [ ]:
ret1, mtx1, dist1, rvecs1, tvecs1, objpoints, imgpoints = calibrate(video=1)
success, rvec1, tvec1 = cv.solvePnP(
    objpoints[0], imgpoints[0], mtx1, dist1, useExtrinsicGuess=False
)

if success:
    R1, _ = cv.Rodrigues(rvec1)

    print("Intrinsic matrix:")
    print(mtx1)

    print("\nRotation Matrix (R):")
    print(R1)

    print("\nTranslation Vector (T):")
    print(tvec1)

Intrinsic matrix:
[[7.05102964e+03 0.00000000e+00 2.24686194e+02]
 [0.00000000e+00 3.87236866e+03 9.20651802e+01]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Rotation Matrix (R):
[[-0.30460514  0.37659377 -0.87486733]
 [-0.64281784  0.59650981  0.48058429]
 [ 0.702852    0.70876877  0.06038124]]

Translation Vector (T):
[[   1.62330402]
 [  -2.41576744]
 [-195.63660301]]


In [12]:
ret2, mtx2, dist2, rvecs2, tvecs2, objpoints, imgpoints = calibrate(video=2)
success, rvec2, tvec2 = cv.solvePnP(objpoints[0], imgpoints[0], mtx2, dist2)

if success:
    R2, _ = cv.Rodrigues(rvec2)

    print("Intrinsic matrix:")
    print(mtx2)

    print("\nRotation Matrix (R):")
    print(R1)

    print("\nTranslation Vector (T):")
    print(tvec2)
mtx2

Intrinsic matrix:
[[ 5.14313452e+03  0.00000000e+00  2.96851077e+02]
 [ 0.00000000e+00  2.28804614e+04 -2.06404461e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Rotation Matrix (R):
[[-0.30460514  0.37659377 -0.87486733]
 [-0.64281784  0.59650981  0.48058429]
 [ 0.702852    0.70876877  0.06038124]]

Translation Vector (T):
[[   92.67835352]
 [  356.11607848]
 [21324.62616693]]


array([[ 5.14313452e+03,  0.00000000e+00,  2.96851077e+02],
       [ 0.00000000e+00,  2.28804614e+04, -2.06404461e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [ ]:
axis_length = 15

world_axes = np.array(
    [
        [0, 0, 0],  # Origin
        [axis_length, 0, 0],  # X-axis
        [0, axis_length, 0],  # Y-axis
        [0, 0, axis_length],  # Z-axis
    ],
    dtype=np.float32,
).reshape(-1, 1, 3)

projected_points, _ = cv.projectPoints(world_axes, rvec2, tvec2, mtx2, dist2)

# Convert points to integer pixel values
projected_points = projected_points.reshape(-1, 2).astype(int)

image = cv.imread("cam2_image30.jpg")

# Draw the axes on the image
origin = tuple(projected_points[0])  # The starting corner
x_axis = tuple(projected_points[1])
y_axis = tuple(projected_points[2])
z_axis = tuple(projected_points[3])

# Draw X-axis (Red)
cv.line(image, origin, x_axis, (0, 0, 255), 3)
cv.putText(image, "X", x_axis, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Draw Y-axis (Green)
cv.line(image, origin, y_axis, (0, 255, 0), 3)
cv.putText(image, "Y", y_axis, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Draw Z-axis (Blue)
cv.line(image, origin, z_axis, (255, 0, 0), 3)
cv.putText(image, "Z", z_axis, cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

# Show the image with axes using OpenCV
cv.imshow("3D World Coordinates Projection", image)
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
ret3, mtx3, dist3, rvecs3, tvecs3, objpoints, imgpoints = calibrate(video=3)
success, rvec3, tvec3 = cv.solvePnP(objpoints[0], imgpoints[0], mtx3, dist3)

if success:
    R3, _ = cv.Rodrigues(rvec3)

    print("Intrinsic matrix:")
    print(mtx3)

    print("\nRotation Matrix (R):")
    print(R3)

    print("\nTranslation Vector (T):")
    print(tvec3)

Intrinsic matrix:
[[6.41707515e+03 0.00000000e+00 4.08261979e+02]
 [0.00000000e+00 8.53231414e+03 2.65466668e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Rotation Matrix (R):
[[ 0.09967506  0.23190391  0.96761845]
 [ 0.10460906  0.96462978 -0.24196349]
 [-0.98950585  0.12533938  0.07189028]]

Translation Vector (T):
[[ -639.08784907]
 [ -245.1702176 ]
 [12666.0616818 ]]


In [14]:
ret4, mtx4, dist4, rvecs4, tvecs4, objpoints, imgpoints = calibrate(video=4)
success, rvec4, tvec4 = cv.solvePnP(objpoints[0], imgpoints[0], mtx4, dist4)
if success:
    R4, _ = cv.Rodrigues(rvec4)

    print("Intrinsic matrix:")
    print(mtx4)

    print("Rotation Matrix (R):")
    print(R4)

    print("\nTranslation Vector (T):")
    print(tvec4)

Intrinsic matrix:
[[1.53344190e+04 0.00000000e+00 3.01189560e+02]
 [0.00000000e+00 1.07105779e+03 2.23532330e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Rotation Matrix (R):
[[-3.92133360e-03 -3.76192872e-03  9.99985235e-01]
 [ 7.00070270e-01 -7.14073955e-01  5.89146220e-05]
 [ 7.14063190e-01  7.00060165e-01  5.43373665e-03]]

Translation Vector (T):
[[   2.44657679]
 [ -36.19193823]
 [-447.62893992]]
